In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import copy
import pandas as pd
import time
import re
import sys
from lib.network_mapper import network_mapper_function
from lib.NR3_OpenDSSvec_splitup import NR3_function


In [2]:
fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'


In [3]:

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(fn, slackidx,Vslack,None,None,None)

[[ 0.        ]
 [ 0.        ]
 [-0.12940177]
 [-0.04063983]
 [-0.12942297]
 [-0.04065241]
 [-0.1294207 ]
 [-0.04064859]
 [-0.1294407 ]
 [-0.04066652]
 [-0.140625  ]
 [-0.046875  ]
 [ 0.        ]
 [ 0.        ]
 [-0.12842133]
 [-0.04061712]
 [-0.12840938]
 [-0.04061062]
 [-0.140625  ]
 [-0.046875  ]
 [-0.12840914]
 [-0.04061013]
 [-0.12841241]
 [-0.04061353]
 [ 0.        ]
 [ 0.        ]
 [-0.13406657]
 [-0.04250134]
 [-0.13407368]
 [-0.04250303]
 [-0.13404099]
 [-0.04249733]
 [-0.1125    ]
 [-0.0375    ]
 [-0.1125    ]
 [-0.0375    ]]
[[ 1.99999718e+00]
 [-5.46744438e-06]
 [-1.00000296e+00]
 [-1.73204529e+00]
 [-1.36890747e+00]
 [ 8.66025404e-01]]
[[ 5.93798101e-04]
 [-1.71028557e-03]
 [ 5.93759661e-04]
 [-1.71029844e-03]
 [ 1.11952387e-03]
 [-2.57462589e-03]
 [-6.13160505e-04]
 [-7.48832781e-06]
 [ 1.66259333e-03]
 [-1.72155599e-03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.05956699e-03]
 [ 8.13292259e-04]
 [-1.05955894e-03]
 [ 8.13331993e-04]
 [-1.35272717e-03]
 [ 1.26096641e-03]
 [

C:\Users\kathl\Desktop\LinDist3Flow\20180601\PYTHON\lib\NR3_OpenDSSvec_splitup.py:71: RuntimeWarning: overflow encountered in matmul
  XNR = XNR - np.linalg.inv(JT.T@JT)@JT.T@FT
C:\Users\kathl\Desktop\LinDist3Flow\20180601\PYTHON\lib\NR3_OpenDSSvec_splitup.py:71: RuntimeWarning: invalid value encountered in matmul
  XNR = XNR - np.linalg.inv(JT.T@JT)@JT.T@FT
C:\Users\kathl\Desktop\LinDist3Flow\20180601\PYTHON\lib\compute_NR3FT_vectorized.py:3: RuntimeWarning: invalid value encountered in matmul
  FTSUBV = (g_SB @ X) - b_SB
C:\Users\kathl\Desktop\LinDist3Flow\20180601\PYTHON\lib\compute_NR3FT_vectorized.py:4: RuntimeWarning: invalid value encountered in matmul
  FTKVL = (G_KVL @ X) - b_KVL
C:\Users\kathl\Desktop\LinDist3Flow\20180601\PYTHON\lib\compute_NR3FT_vectorized.py:11: RuntimeWarning: invalid value encountered in matmul
  + b[0,0,i]
C:\Users\kathl\Desktop\LinDist3Flow\20180601\PYTHON\lib\compute_NR3JT_vectorized.py:8: RuntimeWarning: invalid value encountered in matmul
  r = (2 *

NameError: name 'network' is not defined